# **Rescale number of frames** -> skip evenly distributed number of frames

In [4]:
import os 
import pandas as pd
import numpy as np
import re

In [2]:
os.chdir("/media/data-119")

In [3]:
os.listdir()

['Rat674-20211214_2',
 'rat596_20210705_183011',
 'rat596_20210704_193426',
 'rat596_20210702_175129',
 'nextcloud',
 'rat596_20210703_192113',
 'rat596_20210701_184333',
 'lost+found']

## **July 01**

+ notes: NORMALIZE MVM!!!!!!!!!!!!

### Read ephys timestamp and mvm data, convert ephys to seonds, normalize mvm

In [10]:
#path to SYNC_file & mvm file
ephys_time_path="/media/data-119/rat596_20210701_184333/rat596_20210701_184333tmstp.txt"
mvm_path="/media/data-119/rat596_20210701_184333/rat596_20210701_184333mvm.txt"


#read SYNC_file
e_test = open(ephys_time_path,"r")
ephys_time_test = [[float(x) for x in line.split()] for line in e_test]
e_test.close()

#read mvm_file
m_test = open(mvm_path,"r")
mvm_test = [[float(x) for x in line.split()] for line in m_test]
m_test.close()

#concatenate the list of lists into one list (array)
ephys_time_test = np.concatenate(ephys_time_test)
mvm_test = np.concatenate(mvm_test)

#convert ephys to seconds 
ephys_time_test_sec = [i/2500 for i in ephys_time_test]

In [9]:
len(mvm_test)/len(ephys_time_test_sec)

1.0011209813716409

### Calculate index of lags, number of points between lags

+ it's every 38.5th frame
+ always ~ 0.4 ms = 0.0004 s

+ remove one frame when the lags add up to 40 ms = 0.04 s

-> multiply by factor 100 => 3850

In [12]:
#get the difference between frame in milliseconds
ephys_time_test_sec_diff=np.diff(ephys_time_test_sec)

#get index lags.. time tn-t0
differences_sec=[]
for i in range(len(ephys_time_test_sec_diff)):
    
    #if the diffed time is not exactly 40 ms
    if ephys_time_test_sec_diff[i]-0.04 != 0:
        #can be negative or positive difference
        differences_sec.append((i,ephys_time_test_sec_diff[i]-0.04))
        
#big differences -> from differences select diff of absolute value bigger than 0.0001ms
big_differences_sec=[]
for i in range(len(differences_sec)):
    
    #append tuple if np.abs(difference) > 0.0001
    if np.abs(differences_sec[i][1]) > 0.0001:
        big_differences_sec.append(differences_sec[i])
        
#get the indexes only of big diffs
big_differences_sec_indexes=[big_differences_sec[i][0] for i in range(len(big_differences_sec))]

#np diff of indexes -> after how many frames the difference occur
big_differences_sec_indexes_DIFFS=np.diff(big_differences_sec_indexes)

### How mnay frames are to be deleted?

In [16]:
len(mvm_test)/3850

347.2555844155844

In [17]:
len(mvm_test)-len(ephys_time_test_sec)

1497

## **July 02**

+ notes: NORMALIZE MVM!!!!!!!!!!!!

### Read ephys timestamp and mvm data, convert ephys to seonds, normalize mvm

In [18]:
#path to SYNC_file & mvm file
ephys_time_path="/media/data-119/rat596_20210702_175129/rat596_20210702_175129tmstp.txt"
mvm_path="/media/data-119/rat596_20210702_175129/rat596_20210702_175129mvm.txt"

#read SYNC_file
e_test = open(ephys_time_path,"r")
ephys_time_test = [[float(x) for x in line.split()] for line in e_test]
e_test.close()

#read mvm_file
m_test = open(mvm_path,"r")
mvm_test = [[float(x) for x in line.split()] for line in m_test]
m_test.close()

#concatenate the list of lists into one list (array)
ephys_time_test = np.concatenate(ephys_time_test)
mvm_test = np.concatenate(mvm_test)

#convert ephys to seconds 
ephys_time_test_sec = [i/2500 for i in ephys_time_test]

In [20]:
len(mvm_test)/len(ephys_time_test_sec)

1.0002779308225282

### Calculate index of lags, number of points between lags

+ it's every 38.5th frame
+ always ~ 0.4 ms = 0.0004 s

+ remove one frame when the lags add up to 40 ms = 0.04 s

-> multiply by factor 100 => 3850

In [21]:
#get the difference between frame in milliseconds
ephys_time_test_sec_diff=np.diff(ephys_time_test_sec)

#get index lags.. time tn-t0
differences_sec=[]
for i in range(len(ephys_time_test_sec_diff)):
    
    #if the diffed time is not exactly 40 ms
    if ephys_time_test_sec_diff[i]-0.04 != 0:
        #can be negative or positive difference
        differences_sec.append((i,ephys_time_test_sec_diff[i]-0.04))
        
#big differences -> from differences select diff of absolute value bigger than 0.0001ms
big_differences_sec=[]
for i in range(len(differences_sec)):
    
    #append tuple if np.abs(difference) > 0.0001
    if np.abs(differences_sec[i][1]) > 0.0001:
        big_differences_sec.append(differences_sec[i])
        
#get the indexes only of big diffs
big_differences_sec_indexes=[big_differences_sec[i][0] for i in range(len(big_differences_sec))]

#np diff of indexes -> after how many frames the difference occur
big_differences_sec_indexes_DIFFS=np.diff(big_differences_sec_indexes)

### How mnay frames are to be deleted?

In [22]:
len(mvm_test)/3850

371.11948051948053

In [23]:
len(mvm_test)-len(ephys_time_test_sec)

397

### it doesn't come ONLY  from the big differences.. let's just delete exact num of frames here... spaced evenly

In [25]:
#number of frames in mvm divided by the difference
skip_rate = int(len(mvm_test)/(len(mvm_test)-len(ephys_time_test_sec)))
print(skip_rate)

3599


In [42]:
#create copy from which we delete items
import copy
mvm_test_copy=copy.deepcopy(mvm_test)

In [56]:
mvm_test_copy=[mvm_test[i] for i in range(len(mvm_test)) if (i==0) | (i%skip_rate != 0)]

indexes2delete=[]
for i in range(len(mvm_test)):
    if i == 0:
        continue
    if i%skip_rate == 0:
        del mvm_test_copy[i]

In [37]:
len(indexes2delete) #good number


397

In [40]:
mvm_test_copy = list(mvm_test_copy)
del mvm_test_copy[indexes2delete]

TypeError: list indices must be integers or slices, not list

In [58]:
len(mvm_test_copy)-len(ephys_time_test_sec)

0

In [60]:
np.savetxt("/media/data-119/rat596_20210702_175129/rat596_20210702_175129mvmCOPY.txt",
              mvm_test_copy)